In [ ]:
!pip install pillow requests openai tqdm ultralytics torch torchvision torchaudio boto3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 90.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.3/85.3 kB 7.6 MB/s eta 0:00:00


In [ ]:
import os

from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY_PJ")
os.environ["AWS_S3_BUCKET_NAME"] = userdata.get("AWS_S3_BUCKET_NAME")
os.environ["AWS_ACCESS_KEY_ID"] = userdata.get("AWS_ACCESS_KEY_ID")
os.environ["AWS_SECRET_ACCESS_KEY"] = userdata.get("AWS_SECRET_ACCESS_KEY")
os.environ["AWS_REGION"] = "ap-northeast-2"
os.environ["S3_FOLDER_PREFIX"] = "model_img"

In [ ]:
s3_uploader_code = '''
"""
s3_uploader.py
Virtual Try-on 결과 이미지를 S3에 업로드하고 URL을 반환하는 모듈
"""

import os
import io
import boto3
from pathlib import Path
from typing import Optional, Union
from datetime import datetime
import uuid
from botocore.exceptions import ClientError, NoCredentialsError


class S3ImageUploader:
    """S3에 이미지를 업로드하고 URL을 관리하는 클래스"""

    def __init__(
        self,
        bucket_name: str,
        aws_access_key_id: Optional[str] = None,
        aws_secret_access_key: Optional[str] = None,
        region_name: str = 'ap-northeast-2',
        folder_prefix: str = 'tryon-images'
    ):
        """S3 업로더 초기화"""
        self.bucket_name = bucket_name
        self.folder_prefix = folder_prefix.strip('/')

        try:
            if aws_access_key_id and aws_secret_access_key:
                self.s3_client = boto3.client(
                    's3',
                    aws_access_key_id=aws_access_key_id,
                    aws_secret_access_key=aws_secret_access_key,
                    region_name=region_name
                )
            else:
                self.s3_client = boto3.client('s3', region_name=region_name)

            # 버킷 접근 권한 확인
            self.s3_client.head_bucket(Bucket=bucket_name)

        except NoCredentialsError:
            raise ValueError("AWS 자격 증명을 찾을 수 없습니다.")
        except ClientError as e:
            error_code = e.response['Error']['Code']
            if error_code == '403':
                raise ValueError(f"S3 버킷 '{bucket_name}'에 대한 액세스 권한이 없습니다.")
            elif error_code == '404':
                raise ValueError(f"S3 버킷 '{bucket_name}'을 찾을 수 없습니다.")
            else:
                raise ValueError(f"S3 연결 오류: {e}")

    def _generate_filename(self, product_id: str, extension: str = 'png') -> str:
        """고유한 파일명 생성"""
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        unique_id = str(uuid.uuid4())[:8]
        return f"{self.folder_prefix}/{timestamp}/{product_id}_{unique_id}.{extension}"

    def upload_file(self, file_path: Union[str, Path], product_id: str) -> str:
        """
        로컬 파일을 S3에 업로드하고 공개 URL 반환

        Args:
            file_path: 업로드할 파일 경로
            product_id: 제품 ID

        Returns:
            S3 공개 URL
        """
        file_path = Path(file_path)
        if not file_path.exists():
            raise FileNotFoundError(f"파일을 찾을 수 없습니다: {file_path}")

        extension = file_path.suffix.lstrip('.')
        s3_key = self._generate_filename(product_id, extension)

        try:
            self.s3_client.upload_file(
                str(file_path),
                self.bucket_name,
                s3_key,
                ExtraArgs={
                    'ContentType': 'image/png' if extension.lower() == 'png' else 'image/jpeg',
                    'ACL': 'public-read'
                }
            )

            return f"https://{self.bucket_name}.s3.amazonaws.com/{s3_key}"

        except ClientError as e:
            raise RuntimeError(f"S3 업로드 실패: {e}")


def setup_s3_uploader() -> S3ImageUploader:
    """환경변수에서 설정을 읽어 S3 업로더 생성"""
    bucket_name = os.getenv('AWS_S3_BUCKET_NAME')
    if not bucket_name:
        raise ValueError("환경변수 AWS_S3_BUCKET_NAME이 설정되지 않았습니다.")

    return S3ImageUploader(
        bucket_name=bucket_name,
        aws_access_key_id=os.getenv('AWS_ACCESS_KEY_ID'),
        aws_secret_access_key=os.getenv('AWS_SECRET_ACCESS_KEY'),
        region_name=os.getenv('AWS_REGION', 'ap-northeast-2'),
        folder_prefix=os.getenv('S3_FOLDER_PREFIX', 'tryon-results')
    )
'''

# 파일로 저장
with open('s3_uploader.py', 'w', encoding='utf-8') as f:
    f.write(s3_uploader_code)

print("✅ s3_uploader.py 모듈이 생성되었습니다.")


✅ s3_uploader.py 모듈이 생성되었습니다.


In [ ]:
import sys
from pathlib import Path
from openai import OpenAI

# Python 경로에 현재 디렉토리 추가
if '.' not in sys.path:
    sys.path.append('.')

# 기존 모듈 캐시 삭제
if 's3_uploader' in sys.modules:
    del sys.modules['s3_uploader']

# ========= 사용자 설정 =========
APP_JSON    = Path("app_product.json")   # 제품 리스트
OUT_DIR     = Path("out/tryon_openai")
MODEL_NAME  = "gpt-image-1"              # OpenAI 이미지 편집 모델

# 이번 실행에서 처리할 id들
SELECTED_IDS = [641]

# 캔버스(좌: 모델 / 우: 의류 참고)
SIZE      = (1536, 1024)   # (W, H)
LEFT_BOX  = (0, 0, 768, 1024)
RIGHT_BOX = (768, 0, 1536, 1024)

# 카테고리별 기본 마스크 비율(LEFT_BOX 기준)
CATEGORY_MASK_RATIOS = {
    "upper":     (0.18, 0.18, 0.82, 0.64),
    "lower":     (0.22, 0.55, 0.78, 0.95),
    "outer":     (0.12, 0.12, 0.88, 0.85),
    "onepiece":  (0.18, 0.18, 0.82, 0.90),
    "full":      (0.10, 0.06, 0.90, 0.95),
}
MASK_FEATHER = 16

# 로컬 GPU로 자동 마스크(세그멘테이션+포즈) 사용
USE_LOCAL_GPU_MASK = True
USE_POSE_LANDMARKS = True
YOLO_SEG_MODEL     = "yolov8n-seg.pt"
YOLO_POSE_MODEL    = "yolov8n-pose.pt"
MIN_CONF_KEYPT     = 0.35

# S3 업로드 설정
USE_S3_UPLOAD = True

# OpenAI 클라이언트 초기화
client = OpenAI()

# S3 업로더 초기화
s3_uploader = None
if USE_S3_UPLOAD:
    try:
        from s3_uploader import setup_s3_uploader
        s3_uploader = setup_s3_uploader()
        print(f"✅ S3 업로더 초기화 완료")
        print(f"   버킷: {s3_uploader.bucket_name}")
        print(f"   폴더: {s3_uploader.folder_prefix}")
    except Exception as e:
        print(f"⚠️ S3 업로더 초기화 실패: {e}")
        USE_S3_UPLOAD = False

✅ S3 업로더 초기화 완료
   버킷: elasticbeanstalk-ap-northeast-2-967883357924
   폴더: model_img


In [4]:
# file: tryon_ids_openai.py
import base64
from pathlib import Path
from typing import List, Any, Optional
from PIL import Image, ImageOps, ImageDraw, ImageChops
from openai import OpenAI
from datetime import datetime

def ensure_list(x):
    if x is None: return []
    return x if isinstance(x, list) else [x]

def download_image(url: str) -> Image.Image:
    r = requests.get(url, timeout=60)
    r.raise_for_status()
    return Image.open(io.BytesIO(r.content)).convert("RGBA")

def download_bytes(url: str) -> bytes:
    r = requests.get(url, timeout=60)
    r.raise_for_status()
    return r.content

def fit_into(img: Image.Image, box: tuple, keep_aspect=True, pad_color=(255,255,255,0)) -> Image.Image:
    x1,y1,x2,y2 = box
    w, h = x2-x1, y2-y1
    if keep_aspect:
        img = ImageOps.contain(img, (w,h))
        canvas = Image.new("RGBA", (w,h), pad_color)
        canvas.paste(img, ((w - img.width)//2, (h - img.height)//2), img)
        return canvas
    return img.resize((w,h), Image.LANCZOS)

def make_collage(model_img: Image.Image, garments: List[Image.Image]) -> Image.Image:
    W,H = SIZE
    canvas = Image.new("RGBA", (W,H), (255,255,255,255))
    left_fitted = fit_into(model_img, LEFT_BOX)
    canvas.paste(left_fitted, (LEFT_BOX[0], LEFT_BOX[1]), left_fitted)

    rows = max(1, min(2, len(garments)))
    each_h = (RIGHT_BOX[3] - RIGHT_BOX[1]) // rows
    for i, g in enumerate(garments[:rows], 0):
        slot = (RIGHT_BOX[0], RIGHT_BOX[1] + i*each_h, RIGHT_BOX[2], RIGHT_BOX[1] + (i+1)*each_h)
        fitted = fit_into(g, slot)
        canvas.paste(fitted, (slot[0], slot[1]), fitted)
    return canvas

def _feather(mask: Image.Image, radius: int) -> Image.Image:
    if radius <= 0: return mask
    small = mask.resize((max(1,mask.width//4), max(1,mask.height//4)), Image.BILINEAR)
    return small.resize(mask.size, Image.BILINEAR)

# =============================================================================
# 6. GPU 마스크 및 포즈 관련 함수들
# =============================================================================
def _person_mask_yolov8(model_img_rgba: Image.Image) -> Optional[Image.Image]:
    try:
        from ultralytics import YOLO
        import torch
    except Exception:
        return None
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = YOLO(YOLO_SEG_MODEL)
    crop = model_img_rgba.crop(LEFT_BOX).convert("RGB")
    results = model.predict(crop, imgsz=768, conf=0.25, device=device, verbose=False)
    if not results or len(results[0].masks or []) == 0:
        return None
    import numpy as np
    m = Image.new("L", crop.size, 0)
    for j, cls in enumerate(results[0].boxes.cls.tolist()):
        if int(cls) == 0 and results[0].masks is not None:  # person class
            mask_arr = results[0].masks.data[j].cpu().numpy()
            mask_img = Image.fromarray((mask_arr*255).astype("uint8"), mode="L")
            m = ImageChops.lighter(m, mask_img)
    full = Image.new("L", SIZE, 0)
    full.paste(m, (LEFT_BOX[0], LEFT_BOX[1]))
    return full

def _pose_keypoints_yolo(model_img_rgba: Image.Image) -> Optional[dict]:
    try:
        from ultralytics import YOLO
        import torch, numpy as np
    except Exception:
        return None
    device = "cuda" if torch.cuda.is_available() else "cpu"
    crop = model_img_rgba.crop(LEFT_BOX).convert("RGB")
    model = YOLO(YOLO_POSE_MODEL)
    res = model.predict(crop, imgsz=768, device=device, conf=0.25, verbose=False)
    if not res:
        return None
    r = res[0]
    if r.keypoints is None or r.boxes is None or len(r.keypoints) == 0:
        return None

    areas = (r.boxes.xyxy[:,2]-r.boxes.xyxy[:,0]) * (r.boxes.xyxy[:,3]-r.boxes.xyxy[:,1])
    idx = int((areas).argmax())
    kpts = r.keypoints.xy[idx].cpu().numpy()  # (17,2)
    conf = r.keypoints.conf[idx].cpu().numpy() if r.keypoints.conf is not None else __import__("numpy").ones((kpts.shape[0],))

    KP = {"L_SHOULDER":5,"R_SHOULDER":6,"L_HIP":11,"R_HIP":12,"L_KNEE":13,"R_KNEE":14,"L_ANKLE":15,"R_ANKLE":16}
    def y_of(a,b):
        ys=[]
        for i in (a,b):
            if i < len(kpts) and conf[i] >= MIN_CONF_KEYPT: ys.append(kpts[i][1])
        return int(__import__("numpy").mean(ys)) if ys else None

    y_sh = y_of(KP["L_SHOULDER"], KP["R_SHOULDER"])
    y_hp = y_of(KP["L_HIP"],      KP["R_HIP"])
    y_kn = y_of(KP["L_KNEE"],     KP["R_KNEE"])
    y_an = y_of(KP["L_ANKLE"],    KP["R_ANKLE"])

    x1,y1,x2,y2 = r.boxes.xyxy[idx].cpu().numpy().astype(int).tolist()
    x1 += LEFT_BOX[0]; x2 += LEFT_BOX[0]
    y1 += LEFT_BOX[1]; y2 += LEFT_BOX[1]

    def to_full_y(y): return int(y + LEFT_BOX[1]) if y is not None else None
    return {"shoulders_y":to_full_y(y_sh),"hips_y":to_full_y(y_hp),"knees_y":to_full_y(y_kn),"ankles_y":to_full_y(y_an),"bbox":(x1,y1,x2,y2)}

def make_mask(category: str, base_img: Image.Image) -> Image.Image:
    mask = Image.new("L", SIZE, 0)
    draw = ImageDraw.Draw(mask)

    dyn = _pose_keypoints_yolo(base_img) if USE_POSE_LANDMARKS else None
    def rect(x1,y1,x2,y2): draw.rectangle((int(x1),int(y1),int(x2),int(y2)), fill=255)

    if dyn:
        x1,y1,x2,y2 = dyn["bbox"]
        pad_x = int((x2-x1)*0.08); pad_y = int((y2-y1)*0.05)
        sh = dyn["shoulders_y"] or (LEFT_BOX[1] + int((LEFT_BOX[3]-LEFT_BOX[1])*0.22))
        hp = dyn["hips_y"]      or (LEFT_BOX[1] + int((LEFT_BOX[3]-LEFT_BOX[1])*0.58))
        kn = dyn["knees_y"]     or (LEFT_BOX[1] + int((LEFT_BOX[3]-LEFT_BOX[1])*0.80))
        an = dyn["ankles_y"]    or (LEFT_BOX[1] + int((LEFT_BOX[3]-LEFT_BOX[1])*0.95))

        cat = (category or "upper").lower()
        if cat == "upper":
            rect(max(LEFT_BOX[0], x1+pad_x), max(LEFT_BOX[1], sh - (hp-sh)*0.2),
                 min(LEFT_BOX[2], x2-pad_x), min(LEFT_BOX[3], hp + (hp-sh)*0.05))
        elif cat == "lower":
            rect(max(LEFT_BOX[0], x1+pad_x), max(LEFT_BOX[1], hp - (hp-sh)*0.05),
                 min(LEFT_BOX[2], x2-pad_x), min(LEFT_BOX[3], an))
        elif cat == "outer":
            rect(max(LEFT_BOX[0], x1+int(pad_x*0.7)), max(LEFT_BOX[1], sh - (hp-sh)*0.6),
                 min(LEFT_BOX[2], x2-int(pad_x*0.7)), min(LEFT_BOX[3], kn))
        elif cat in ("onepiece","dress"):
            rect(max(LEFT_BOX[0], x1+pad_x), max(LEFT_BOX[1], sh - (hp-sh)*0.2),
                 min(LEFT_BOX[2], x2-pad_x), min(LEFT_BOX[3], max(kn, hp + (hp-sh))))
        else:  # full
            rect(max(LEFT_BOX[0], x1+pad_x), max(LEFT_BOX[1], y1+pad_y),
                 min(LEFT_BOX[2], x2-pad_x), min(LEFT_BOX[3], an))
    else:
        rx1,ry1,rx2,ry2 = CATEGORY_MASK_RATIOS.get((category or "upper").lower(), CATEGORY_MASK_RATIOS["upper"])
        lx1,ly1,lx2,ly2 = LEFT_BOX
        rect(lx1 + (lx2-lx1)*rx1, ly1 + (ly2-ly1)*ry1, lx1 + (lx2-lx1)*rx2, ly1 + (ly2-ly1)*ry2)

    if USE_LOCAL_GPU_MASK:
        person = _person_mask_yolov8(base_img)
        if person is not None:
            mask = ImageChops.multiply(mask, person)

    return _feather(mask, MASK_FEATHER)

# =============================================================================
# 7. OpenAI 이미지 편집 + S3 업로드 함수
# =============================================================================
def call_openai_edit(base_img, mask_img, prompt, out_path):
    """OpenAI 이미지 편집 + S3 업로드"""
    # 1) 마스크를 RGBA로 만들고, 알파 채널에 마스크를 복사
    mask_L = mask_img.convert("L")
    mask_rgba = mask_L.convert("RGBA")
    mask_rgba.putalpha(mask_L)

    # 2) BytesIO에 저장 + name 부여
    base_buf = io.BytesIO()
    base_img.save(base_buf, format="PNG")
    base_buf.seek(0)
    base_buf.name = "base.png"

    mask_buf = io.BytesIO()
    mask_rgba.save(mask_buf, format="PNG")
    mask_buf.seek(0)
    mask_buf.name = "mask.png"

    # 3) OpenAI API 호출
    resp = client.images.edit(
        model=MODEL_NAME,
        image=base_buf,
        mask=mask_buf,
        prompt=prompt,
        size="1024x1024",
        n=1,
    )

    # 4) 로컬 저장
    out_path.parent.mkdir(parents=True, exist_ok=True)
    b64 = resp.data[0].b64_json
    out_path.write_bytes(base64.b64decode(b64))
    print(f"💾 로컬 저장 완료: {out_path}")

    # 5) S3 업로드
    if USE_S3_UPLOAD and s3_uploader:
        try:
            product_id = out_path.stem.split('_')[0]
            s3_url = s3_uploader.upload_file(out_path, product_id)
            print(f"📤 S3 업로드 완료: {s3_url}")
        except Exception as e:
            print(f"❌ S3 업로드 실패: {e}")

    return str(out_path)


# =============================================================================
# 8. 모델 및 제품 데이터 로딩 함수들
# =============================================================================
def load_models_by_gender(path: Path) -> dict:
    """model_list.json을 로드해 성별별 대표 이미지 URL 딕셔너리로 정규화"""
    if not path.exists():
        raise FileNotFoundError(f"모델 JSON 없음: {path}")

    raw = path.read_text(encoding="utf-8").strip()
    try:
        obj = json.loads(raw)
    except json.JSONDecodeError:
        obj = [json.loads(l) for l in raw.splitlines() if l.strip()]

    db = {}

    def _norm_gender(g: str) -> str | None:
        g = (g or "").strip().lower()
        if g in ("female","f","여","여자","woman","girl","w"): return "female"
        if g in ("male","m","남","남자","man","boy"):         return "male"
        if g in ("default","any","*"):                        return "default"
        return None

    def _extract_url(v) -> str | None:
        if isinstance(v, dict):
            return v.get("model_url") or v.get("image_url")
        if isinstance(v, str):
            return v
        return None

    if isinstance(obj, dict):
        for g, v in obj.items():
            key = _norm_gender(g)
            url = _extract_url(v)
            if key and url:
                db[key] = url
    else:
        for it in obj:
            key = _norm_gender(it.get("gender"))
            url = it.get("model_url") or it.get("image_url")
            if key and url:
                db[key] = url

    return db

def resolve_model_url_by_gender(item: dict, models_by_gender: dict) -> Optional[str]:
    g = str(item.get("gender") or item.get("model_gender") or "").strip().lower()
    key = "female" if g in ("female","f","여","여자","woman","girl","w") else "male" if g in ("male","m","남","남자","man","boy") else None
    if key and key in models_by_gender: return models_by_gender[key]
    if "default" in models_by_gender:   return models_by_gender["default"]
    return models_by_gender.get("female") or models_by_gender.get("male")

def load_items_by_ids(path: Path, selected_ids: List[Any]) -> List[dict]:
    raw = path.read_text(encoding="utf-8")
    try:
        data = json.loads(raw)
        if isinstance(data, dict): data = [data]
    except json.JSONDecodeError:
        data = [json.loads(l) for l in raw.splitlines() if l.strip()]
    sids = {str(x) for x in (selected_ids or [])}
    picked = [it for it in data if str(it.get("id") or it.get("external_id")) in sids]
    if not picked:
        raise ValueError(f"id {sorted(sids)} 항목을 찾지 못했습니다.")
    return picked

# =============================================================================
# 9. 결과 확인 함수들
# =============================================================================
def get_s3_urls_from_results(results_dir="out/tryon_openai"):
    """생성된 결과에서 S3 URL들을 수집"""
    results_dir = Path(results_dir)
    s3_urls = {}

    for info_file in results_dir.glob("*_info.json"):
        try:
            data = json.loads(info_file.read_text())
            product_id = data.get("product_id")
            s3_url = data.get("s3_url")

            if product_id and s3_url:
                s3_urls[product_id] = {
                    "url": s3_url,
                    "folder": data.get("s3_folder"),
                    "local_path": data.get("local_path"),
                    "timestamp": data.get("timestamp")
                }
        except:
            continue

    return s3_urls


def main():
    if not os.getenv("OPENAI_API_KEY"):
        raise RuntimeError("환경변수 OPENAI_API_KEY 필요")

    # GPU 도구 확인(없으면 자동 비활성화)
    if USE_LOCAL_GPU_MASK or USE_POSE_LANDMARKS:
        try:
            import torch, ultralytics  # noqa
        except Exception as e:
            print(f"[경고] 로컬 GPU 마스크/포즈 비활성화: {e}")
            globals()["USE_LOCAL_GPU_MASK"] = False
            globals()["USE_POSE_LANDMARKS"] = False

    # 제품 데이터 로드 (실제 URL은 무시)
    items = load_items_by_ids(APP_JSON, SELECTED_IDS)
    OUT_DIR.mkdir(parents=True, exist_ok=True)

    for item in items:
        pid = item.get("id") or item.get("external_id")

        # 실제 garment_urls 무시 → 더미 이미지 2개 생성
        garments = []
        for i, color in enumerate(["red", "blue"]):
            g = Image.new("RGBA", (512, 512), color=color)
            garments.append(g)

        # 모델 이미지도 더미 생성 (녹색)
        model_img = Image.new("RGBA", (512, 1024), color="green")

        # 콜라주 & 마스크
        collage  = make_collage(model_img, garments)
        category = str(item.get("category") or "upper").lower()
        mask_img = make_mask(category, collage)

        # 프롬프트 (더미)
        prompt = "왼쪽 사람에 녹색 옷을 입히고 오른쪽 의류(빨강, 파랑)를 참조하여 착장처럼 보이게"

        # 생성 → 이미지 파일 저장 + S3 업로드
        out_path = OUT_DIR / f"{pid}_tryon.png"
        try:
            saved = call_openai_edit(collage, mask_img, prompt, out_path)
            print(f"[OK] id={pid} → {saved}")
        except Exception as e:
            print(f"[ERROR] id={pid} 편집 실패: {e}")



# =============================================================================
# 11. 실행
# =============================================================================
if __name__ == "__main__":
    main()


RuntimeError: 환경변수 OPENAI_API_KEY 필요

In [9]:
# file: tryon_looks_sdxl_s3_colab.py
import os, io, json, re, datetime, asyncio, time, requests
from pathlib import Path
from typing import List, Any, Dict, Optional
from PIL import Image
from zoneinfo import ZoneInfo
from dotenv import load_dotenv

import torch
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel
import cv2, numpy as np

# Colab/Jupyter 이벤트 루프 충돌 방지
import nest_asyncio
nest_asyncio.apply()

load_dotenv()

from huggingface_hub import login
login(os.getenv("HF_TOKEN"))  # 또는 직접 문자열 토큰 입력


# =========================
# 환경변수
# =========================
SIZE_W, SIZE_H = 512, 512
MAX_REFS = 6
LOOKS_JSON = Path(os.getenv("LOOKS_JSON_PATH", "S3/app_product_test.json"))

AWS_S3_BUCKET  = os.getenv("AWS_S3_BUCKET_NAME")
AWS_REGION     = os.getenv("AWS_S3_REGION", "ap-northeast-2")
AWS_S3_PREFIX  = os.getenv("AWS_S3_PREFIX", "tryon")

# ===== 모델 조합 =====
# HF 토큰으로 접근 승인 받은 SDXL base를 쓰려면 아래 라인 사용
BASE_MODEL = "stabilityai/stable-diffusion-xl-base-1.0"   # (gated) 로그인 필요
CONTROL_MODEL = "lllyasviel/sd-controlnet-canny"

device = "cuda" if torch.cuda.is_available() else "cpu"

# ===== 파이프라인 로드 (1회) =====
controlnet = ControlNetModel.from_pretrained(
    CONTROL_MODEL,
    torch_dtype=torch.float16 if device == "cuda" else torch.float32,
)
pipe = StableDiffusionControlNetPipeline.from_pretrained(
    BASE_MODEL,
    controlnet=controlnet,
    torch_dtype=torch.float16 if device == "cuda" else torch.float32,
    # safety_checker=None  # ❌ 배포용 아님: 기본값 유지(경고 안 뜨게)
).to(device)

# ---------- JSON → 룩 정규화 ----------
KNOWN_PART_KEYS = ["top","bottom","outer","onepiece","dress","bag","shoes","acc","accessory"]

def normalize_look(look: Dict) -> Dict:
    garment_urls: List[str] = []
    ref_ids: List[str] = []
    def add_part(part: Any):
        if not isinstance(part, dict): return
        url = part.get("image_url")
        pid = part.get("search_history_product_id")
        if url: garment_urls.append(url)
        if pid is not None: ref_ids.append(str(pid))

    for k in KNOWN_PART_KEYS:
        if k in look:
            v = look[k]
            if isinstance(v, list):
                for it in v: add_part(it)
            else:
                add_part(v)
    raw_look_id = look.get("look_id") or look.get("id") or "_".join(ref_ids) or "look"
    return {
        "look_id": str(raw_look_id),
        "garment_urls": garment_urls[:MAX_REFS],
        "ref_ids": ref_ids,
        "meta": {"look_style": look.get("look_style")}
    }

def load_all_looks(path: Path) -> List[Dict]:
    obj = json.loads(path.read_text(encoding="utf-8"))
    if isinstance(obj, dict) and isinstance(obj.get("results"), list):
        data = obj["results"]
    elif isinstance(obj, list):
        data = obj
    elif isinstance(obj, dict):
        data = [obj]
    else:
        raise RuntimeError("룩 JSON 형식 오류")
    looks = [normalize_look(look) for look in data]
    return [lk for lk in looks if lk["garment_urls"]]

# ---------- 403 우회 다운로드 유틸 ----------
from urllib.parse import urlparse
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

def _make_session() -> requests.Session:
    s = requests.Session()
    retry = Retry(
        total=3, backoff_factor=0.6,
        status_forcelist=[403, 408, 429, 500, 502, 503, 504],
        allowed_methods=["GET", "HEAD"]
    )
    adapter = HTTPAdapter(max_retries=retry, pool_connections=20, pool_maxsize=20)
    s.mount("http://", adapter)
    s.mount("https://", adapter)
    return s

SESSION = _make_session()

def _headers_for(url: str) -> Dict[str, str]:
    o = urlparse(url)
    referer = f"{o.scheme}://{o.netloc}/"
    return {
        "User-Agent": ("Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                       "AppleWebKit/537.36 (KHTML, like Gecko) "
                       "Chrome/115.0 Safari/537.36"),
        "Referer": referer,
        "Accept": "image/avif,image/webp,image/apng,image/*,*/*;q=0.8",
        "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8",
        "Connection": "keep-alive",
    }

def download_image(url: str, timeout: int = 30) -> Optional[Image.Image]:
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                      "AppleWebKit/537.36 (KHTML, like Gecko) "
                      "Chrome/120.0.0.0 Safari/537.36",
        "Referer": url.split("/web/")[0],  # mullang.com 같은 도메인
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
    }
    try:
        r = requests.get(url, headers=headers, timeout=timeout)
        r.raise_for_status()
        return Image.open(io.BytesIO(r.content)).convert("RGB")
    except Exception as e:
        print(f"[WARN] 이미지 다운로드 실패: {url} ({e})")
        return None

# ---------- Canny 변환 ----------
def make_canny_image(pil_img: Image.Image) -> Image.Image:
    arr = np.array(pil_img.convert("RGB"))
    gray = cv2.cvtColor(arr, cv2.COLOR_RGB2GRAY)
    edges = cv2.Canny(gray, 100, 200)
    edges_rgb = cv2.cvtColor(edges, cv2.COLOR_GRAY2RGB)
    return Image.fromarray(edges_rgb)

# ---------- 프롬프트 ----------
def build_prompt(garment_urls: List[str], look_style: Optional[str]) -> str:
    refs = ", ".join(garment_urls)
    style_hint = f" Style: {look_style}." if look_style else ""
    return (
        "A full-body photorealistic studio photo of a fashion model. "
        "The model must be wearing these exact garments without alteration: "
        f"{refs}."
        f"{style_hint} Natural face, realistic folds/shadows, minimal background."
    )

# ---------- 이미지 생성 ----------
async def generate_image(garment_urls: List[str], prompt: str) -> Optional[bytes]:
    loop = asyncio.get_event_loop()
    def _run():
        # 여러 장 중 접근 가능한 것만 수집
        imgs: List[Image.Image] = []
        for u in garment_urls:
            if not u:
                continue
            img = download_image(u)
            if img is not None:
                imgs.append(img)
        if not imgs:
            print("[ERROR] 다운로드 가능한 의류 이미지가 없음")
            return None

        # 현재는 첫 장만 ControlNet에 사용 (여러 장 멀티 Control은 확장 가능)
        control_img = make_canny_image(imgs[0])

        result = pipe(
            prompt=prompt,
            image=control_img,
            num_inference_steps=20,     # Colab T4 타협값
            guidance_scale=7.0,
            width=SIZE_W,
            height=SIZE_H,
        ).images[0]

        buf = io.BytesIO()
        result.save(buf, format="PNG")
        buf.seek(0)
        return buf.getvalue()
    return await loop.run_in_executor(None, _run)

# ---------- S3 업로더 ----------
def _sanitize_key_part(s: str) -> str:
    s = s.strip().replace(" ", "_")
    return re.sub(r"[^A-Za-z0-9._/-]+", "-", s)

class S3Uploader:
    def __init__(self, bucket: str, region: str, prefix: str = "tryon"):
        import boto3
        self.bucket = bucket
        self.region = region
        self.prefix = (prefix or "").strip("/")
        self.s3 = boto3.client("s3", region_name=region)

    def put_bytes(self, data: bytes, key: str, content_type="image/png") -> Dict[str,str]:
        self.s3.put_object(
            Bucket=self.bucket, Key=key, Body=data,
            ContentType=content_type, ACL="public-read"
        )
        return {
            "url": f"https://{self.bucket}.s3.{self.region}.amazonaws.com/{key}"
        }

    def build_key(self, filename: str, look_id: str) -> str:
        date = datetime.datetime.now(ZoneInfo("Asia/Seoul")).strftime("%Y%m%d")
        safe_look = _sanitize_key_part(look_id)
        safe_file = _sanitize_key_part(filename)
        return f"{self.prefix}/{date}/{safe_look}/{safe_file}"

# ---------- 실행 ----------
async def process_look(look, idx, uploader):
    look_id = look["look_id"]
    garment_urls = look["garment_urls"]
    print(f"[INFO] look#{idx} ({look_id}) 시작")
    prompt = build_prompt(garment_urls, look["meta"].get("look_style"))

    t0 = time.time()
    img_bytes = await generate_image(garment_urls, prompt)
    if not img_bytes:
        print(f"[ERROR] look#{idx} ({look_id}) 생성 실패")
        return None

    out_name = f"look_{idx:03d}_{_sanitize_key_part(look_id)}.png"
    s3_key = uploader.build_key(out_name, look_id)
    res = uploader.put_bytes(img_bytes, s3_key)
    print(f"[OK] {look_id} → {res['url']}  (⏱ {time.time()-t0:.2f}s)")
    return res

async def main_async():
    looks = load_all_looks(LOOKS_JSON)[:5]
    uploader = S3Uploader(AWS_S3_BUCKET, AWS_REGION, AWS_S3_PREFIX)
    tasks = [process_look(look, idx, uploader) for idx, look in enumerate(looks, start=1)]
    await asyncio.gather(*tasks)

# ⬇️ Colab/Jupyter: asyncio.run() 쓰지 말고 await
start = time.time()
await main_async()
print(f"총 소요시간: {time.time()-start:.2f} 초")


Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

You have disabled the safety checker for <class 'diffusers.pipelines.controlnet.pipeline_controlnet.StableDiffusionControlNetPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


OutOfMemoryError: CUDA out of memory. Tried to allocate 14.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 4.12 MiB is free. Process 161252 has 14.73 GiB memory in use. Of the allocated memory 14.38 GiB is allocated by PyTorch, and 259.58 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [5]:
!pip install diffusers==0.31.0 transformers accelerate safetensors boto3 opencv-python-headless

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 41.7 MB/s eta 0:00:00
  Attempting uninstall: diffusers
    Found existing installation: diffusers 0.35.1
    Uninstalling diffusers-0.35.1:
      Successfully uninstalled diffusers-0.35.1
